# Xbox Game Pass - Análise de Dados do Portfólio

# Introdução
O Xbox Game Pass é um serviço de assinatura que oferece acesso a uma biblioteca de centenas de jogos de diversos gêneros. Neste projeto, explorarei o portfólio do Xbox Game Pass com base na última atualização do conjunto de dados, em 2022. Também buscarei aprimorar o conjunto de dados adicionando gêneros de jogos, proporcionando uma análise mais aprofundada.

# Objetivos
1. Aprimorar o conjunto de dados adicionando gêneros de jogos, permitindo uma análise mais abrangente.
2. Realizar uma análise exploratória de dados para obter insights sobre os gêneros de jogos e o desempenho do portfólio do Xbox Game Pass.
3. Visualizar os dados para garantir que os resultados sejam claros, acessíveis e fáceis de interpretar.

In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Imports necessary libraries for data manipulation and visualisation

# 1. Gêneros de Jogos

1. A expansão do conjunto de dados para incluir gêneros de jogos foi alcançada carregando o conjunto de dados original (arquivo CSV) em um LLM (Grok) e solicitando que ele pesquisasse o gênero de cada jogo e, em seguida, criasse uma nova coluna para todos os mais de 400 jogos.
2. O Grok conseguiu pesquisar e executar essa tarefa atribuindo a cada jogo da lista o gênero correto, incluindo subgêneros.
3. Uma limpeza de dados foi realizada no Planilhas Google, separando os gêneros primários dos subgêneros, preparando o conjunto de dados para análise.

In [ ]:
# New enhanced CSV with seperated genres
xbox = pd.read_csv('/kaggle/input/enhanced-xbox-game-pass-genres/Enhanced_Gamepass_Games_v2.csv')

xbox.head()

# 2. Análise de Dados e Visualizações

## Avaliações por Gênero

**Quais são as avaliações médias por gênero? (Gênero principal)**

In [ ]:
# Group by Main_Genre and calculate avg rating and count of games
main_genre_stats = xbox.groupby('Main_Genre').agg({
    'RATING': 'mean',
    'GAME': 'count'
}).rename(columns={'RATING': 'Average_Rating', 'GAME': 'Game_Count'}).reset_index()

# Sort by Average_Rating in descending order
main_genre_stats = main_genre_stats.sort_values('Average_Rating', ascending=False)

main_genre_stats['Average_Rating'] = main_genre_stats['Average_Rating'].round(2)

main_genre_stats.head(25)

Embora gêneros como **Metroidvania** pareçam ter a classificação mais alta, este não é um indicador confiável, pois se baseia em apenas um jogo, em comparação com **Ação**, que inclui 182 jogos.

Vamos combinar os subgêneros e contá-los para obter uma visão melhor.

In [ ]:
# Clean columns (strip whitespace to fix dupelicates)
for col in ['Main_Genre', 'second_genre', 'third_genre', 'fourth_genre']:
    xbox[col] = xbox[col].str.strip()

# Combine the genre columns into a single column
genres_combined = pd.melt(
    xbox,
    id_vars=['GAME', 'RATING', 'GAMERS'],
    value_vars=['Main_Genre', 'second_genre', 'third_genre', 'fourth_genre'],
    value_name='Genre'
).dropna(subset=['Genre'])

# Group by Genre and calculate average rating and count
all_genre_stats = genres_combined.groupby('Genre').agg({
    'RATING': 'mean',
    'GAME': 'count'
}).rename(columns={'RATING': 'Average_Rating', 'GAME': 'Game_Count'}).reset_index()

all_genre_stats['Average_Rating'] = all_genre_stats['Average_Rating'].round(2)
all_genre_stats = all_genre_stats.sort_values('Game_Count', ascending=False)

all_genre_stats.head(25)

Vamos visualizar estes resultados:

In [ ]:
# Create an interactive bar plot for top 15 genres by Game_Count using Plotly
fig = px.bar(all_genre_stats.head(15), 
             x='Average_Rating', 
             y='Genre', 
             color='Game_Count',  # Use Game_Count for color variation
             color_continuous_scale='Viridis',
             title='Average Rating by Genre',
             labels={'Average_Rating': 'Average Rating (1-5)', 'Genre': 'Genre', 'Game_Count': 'Game Count'},
             category_orders={'Genre': all_genre_stats.head(15).sort_values('Average_Rating', ascending=False)['Genre'].tolist()},
             height=600,  # Adjust height
             width=800)   # Adjust width

# Update layout
fig.update_layout(
    xaxis_title="Average Rating (1-5)",
    yaxis_title="Genre",
    coloraxis_colorbar_title="Game Count",
    showlegend=True
)

# Show the interactive plot
fig.show(renderer='iframe')

Os gêneros **Ação** e **Aventura** têm um grande número de jogos (contagem de jogos) e uma classificação média alta, sugerindo que esses são os gêneros mais jogados e populares, juntamente com **RPG**.

## Avaliações vs. Taxa de Conclusão

**Gêneros com maior porcentagem de conclusão (COMP%) têm avaliações melhores, indicando a satisfação do jogador?**

Vou me concentrar no gênero principal ou "gênero principal" do jogo para esta análise (em vez de incluir subgêneros).

In [ ]:
# Group by Main_Genre and calculate mean COMP % and RATING
genre_comp = xbox.groupby('Main_Genre').agg({
    'COMP %': 'mean',
    'RATING': 'mean',
    'GAME': 'count'  # Count of games per genre for context
}).rename(columns={'GAME': 'Game_Count'}).reset_index()

# Sort by COMP % to identify genres with high completion rates
genre_comp = genre_comp.sort_values('COMP %', ascending=False)

genre_comp['COMP %'] = genre_comp['COMP %'].round(2)
genre_comp['RATING'] = genre_comp['RATING'].round(2)

genre_comp.head(10)

**Aventura** e **Arcade** têm as maiores taxas de conclusão, mas como isso fica quando levamos em consideração as classificações dos jogos?

In [ ]:
# Interactive scatter plot with hover
fig = px.scatter(genre_comp, x='COMP %', y='RATING', color='Main_Genre',
                 title='Average Completion Rate vs. Rating by Genre',
                 labels={'COMP %': 'Average Completion Rate (%)', 'RATING': 'Average Rating (out of 5)'},
                 hover_data=['Main_Genre'])  # Hover shows genre

fig.update_layout(
    width=800,
    height=600,
    showlegend=True
)
fig.show(renderer='iframe')

Podemos observar alguns casos excepcionais:

1. Arcade
2. Aventura
3. Plataforma
4. Simulação
5. Metroidvania

Jogos com maiores taxas de conclusão têm classificações melhores?

Em média, jogos com maior porcentagem de conclusão **não** indicam classificações mais altas, com exceção do gênero **Plataforma**, que tem a terceira maior taxa de conclusão *e* a terceira maior classificação.

Por fim, vamos analisar:

## Distribuição de Jogadores por Gênero

**Quais gêneros têm o maior engajamento dos jogadores?** Vamos destacar os gêneros mais populares no Game Pass, potencialmente orientando a estratégia de conteúdo.

> Observação sobre a contagem de jogadores: jogos multigênero (por exemplo, ação e aventura) somam a contagem total de jogadores a cada gênero, resultando em contagem dupla. Isso mostra a exposição ao gênero, não jogadores únicos por gênero.

In [ ]:
# Ensure GAMERS is numeric in the original DataFrame before melting
xbox['GAMERS'] = pd.to_numeric(xbox['GAMERS'].str.replace(',', ''), errors='coerce')

# Recreate genres_combined with proper handling to avoid GAMERS duplication
genres_combined_gamers = pd.melt(
    xbox,
    id_vars=['GAME', 'GAMERS'],  # Focus on GAME and GAMERS
    value_vars=['Main_Genre', 'second_genre', 'third_genre', 'fourth_genre'],
    value_name='Genre'
).dropna(subset=['Genre'])

# Group by Genre and calculate total gamers and unique game count
genre_gamers_dist = genres_combined_gamers.groupby('Genre').agg({
    'GAMERS': 'sum',  # Sum the numeric GAMERS values
    'GAME': 'nunique'  # Count unique games to avoid overcounting
}).rename(columns={'GAMERS': 'Total_Gamers', 'GAME': 'Game_Count'}).reset_index()

# Ensure Total_Gamers is numeric and handle any remaining issues
genre_gamers_dist['Total_Gamers'] = pd.to_numeric(genre_gamers_dist['Total_Gamers'], errors='coerce')

# Round Total_Gamers to the nearest million for readability
genre_gamers_dist['Total_Gamers_Millions'] = (genre_gamers_dist['Total_Gamers'] / 1_000_000).round(0) * 1_000_000

# Sort by Total_Gamers in descending order
genre_gamers_dist = genre_gamers_dist.sort_values('Total_Gamers', ascending=False)

# Display the top 15 genres with formatted values
genre_gamers_dist.head(15)

In [ ]:
# Create an interactive bar plot for Player Distribution Across Genres
fig = px.bar(genre_gamers_dist.head(10), 
             x='Total_Gamers',
             y='Genre',
             color='Game_Count',  # Use Game_Count for color variation
             color_continuous_scale='Viridis',
             title='Player Exposure Across Genres (Top 10 by Total Gamers)',
             labels={'Total_Gamers': 'Total Gamers (Thousands)', 'Genre': 'Genre', 'Game_Count': 'Game Count'},
             height=600,
             width=800)

fig.update_layout(
    xaxis_title="Total Gamers",
    yaxis_title="Genre",
    coloraxis_colorbar_title="Game Count",
    showlegend=True
)

fig.show(renderer='iframe')

Os gêneros **Ação** e **Aventura** têm a maior exposição de jogadores. Uma contagem maior de jogos está fortemente correlacionada a uma maior distribuição de jogadores.

# Resumo

> 1. Os gêneros **Ação** e **Aventura** têm um alto número de jogos e uma alta classificação média, sugerindo que são os gêneros mais populares e mais bem alcançados no Xbox Game Pass.
> 2. Jogos com uma porcentagem de conclusão maior **não** indicam classificações mais altas, com exceção dos jogos de **Plataforma**.
> 3. Jogos marcados como **Ação** e **Aventura** representam cerca de 50% dos jogos no Game Pass pelo total de jogadores (40 milhões).

## Reflexão

Para análises futuras deste conjunto de dados, gostaria de entender:

1. Gêneros de jogos populares recebem avaliações mais altas devido ao seu apelo mais amplo? Ou jogos de nicho (menor número de jogadores) recebem avaliações mais altas de fãs menores e mais dedicados?
2. Existe uma correlação entre o tempo de conclusão (horas) e avaliações mais altas ou mais baixas?
3. Gostaria de refatorar o código implementando uma fusão combinada para reduzir a duplicação de código.

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vines666/enhanced-xbox-game-pass-genres")

print("Path to dataset files:", path)

ModuleNotFoundError: No module named 'kagglehub'